In [28]:
import tqdm
import sys
import daisy.utils
import pandas as pd
from daisy.utils.metrics import *

sys.path.append('../daisyRec')

### Version

daisyRec version 1.1.2

### Data preparation

In [5]:
df_pred = pd.read_csv('./preds_full.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.relevance.value_counts()

0.0    21307932
4.5       25887
5.0       12924
Name: relevance, dtype: int64

In [7]:
recommendations = {}

for user, relevance in zip(df_pred_new.user_id, df_pred_new.relevance):
    if user in recommendations:
        recommendations[user] += [relevance]
    else:
        recommendations[user] = [relevance]

In [8]:
recommendations_top = {}

for user in tqdm.tqdm(recommendations):
    recommendations_top[user] = recommendations[user][:20]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2733/2733 [00:00<00:00, 202084.35it/s]


In [9]:
recommendations_top = {}

for user in tqdm.tqdm(df_pred_new.user_id.unique()):
    recommendations_top[user] = df_pred_new[df_pred_new.user_id == user].sort_values('score', ascending=False).relevance[:20]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2733/2733 [01:15<00:00, 36.06it/s]


In [10]:
recommendations_top_bin = {k: [1 if x > 0 else 0 for x in v] for k,v in recommendations_top.items()}

In [11]:
gt_recommendations = {}

for user, item_id in zip(df_test.user_id, df_test.item_id):
    if user in gt_recommendations:
        gt_recommendations[user] += [item_id]
    else:
        gt_recommendations[user] = [item_id] 

### Evaluation metrics

In [16]:
results = pd.DataFrame()

In [29]:
#map_at_k FAILED
results.at["MAP@20","value"] = map_at_k(recommendations_top)

IndexError: invalid index to scalar variable.

Let's fix this small bug:

In [30]:
def map_at_k(rs):
    """
    Mean Average Precision calculation method
    Parameters
    ----------
    rs : Dict, {user : rank items} for test set
    Returns
    -------
    m_a_p : float, MAP value
    """
    m_a_p = np.mean([ap(r) for r in rs.values()])
    return m_a_p

In [31]:
results.at["MAP@20","value"] = map_at_k(recommendations_top)

In [17]:
results.at["HitRate@20","value"] = hr_at_k(recommendations_top, gt_recommendations)

In [21]:
results.at["Precision@20","value"] = np.mean([precision_at_k(x, 20) for x in list(recommendations.values())])

In [24]:
results.at["NDCG@20","value"] = np.mean([
    ndcg_at_k(
        df_pred_new[df_pred_new.user_id == user].relevance.tolist(), 
        20
    ) 
    for user in recommendations
])

In [18]:
results.at["Recall@20","value"] = recall_at_k(recommendations, gt_recommendations, 20)

In [25]:
results.at["MRR@20","value"] = mrr_at_k(recommendations, 20)

In [32]:
#auc_at_k FAILED
results.at["AUC@20","value"] = auc_at_k(recommendations_top)

NameError: name 'pred_all' is not defined

Bug: https://github.com/AmazingDD/daisyRec/blob/1b7d693f07b892f26eb026d416d86a59a6c3e3a0/daisy/utils/metrics.py#L202

### Results

In [33]:
results

,value
HitRate@20,0.474570
Recall@20,0.096321
Precision@20,0.057519
MAP@20,0.022971
NDCG@20,0.093264
MRR@20,0.274882
